In [1]:
import torch

# Load the GNN embeddings from .pt file
gnn_embeddings = torch.load("node_embeddings.pt")

# Check the shape
print(gnn_embeddings.shape)  # Example: (num_dialogues, embedding_dim)


torch.Size([39283, 64])


In [12]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Convert GNN embeddings to NumPy
embeddings_np = gnn_embeddings.numpy()

# Create FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

# Load a text embedding model (to embed new queries)
text_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve_similar_questions(query, k=5):
    query_embedding = text_model.encode([query])  # Convert text to embedding
    D, I = index.search(query_embedding, k)  # Retrieve top-k similar dialogues
    return I[0]  # Return indices of most relevant dialogues




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a pre-trained LLM (e.g., Mistral, LLaMA)
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Example question
user_question = "How do neural networks generalize?"

# Retrieve structurally similar dialogues
retrieved_indices = retrieve_similar_questions(user_question)

# Modify input prompt
structured_input = modify_llm_input(user_question, retrieved_indices, dialogues)

# Tokenize and generate response
input_ids = tokenizer(structured_input, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=200)
response = tokenizer.decode(output[0], skip_special_tokens=True)

print("LLM Response:", response)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]